In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from keras.models import load_model
import tensorflow as tf
new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/drowsy_detection/vggRetrain/vgg16Retrain_5epoch')

In [ ]:
!pip install mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 7.6 MB/s 


In [ ]:
import mtcnn
# print version
print(mtcnn.__version__)

In [ ]:
# Extract only eye region from image - haarcascade
import cv2
eyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_eye.xml")
leyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_lefteye_2splits.xml")

In [ ]:
class_names = {'Closed': 0, 'Open': 1, 'no_yawn': 2, 'yawn': 3}


In [ ]:
reversed_dictionary = dict(map(reversed, class_names.items()))

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import image
import os


In [ ]:
def detect_faces_from_img(filename):
    pixels = plt.imread(filename)
    detector = mtcnn.MTCNN()
    faces = detector.detect_faces(pixels)
    return faces[0]


In [ ]:
def crop_faces(face,filename):
  img = plt.imread(filename)
  x, y, w, h = face['box']
  cropimg = img[y:y+h, x:x+w]
  return cropimg


In [ ]:
def get_prediction_eye(filename,model):
  face = detect_faces_from_img(filename)
  cropimg = crop_faces(face,filename)
  imageCrop = np.expand_dims(cropimg, axis=0)
  imageCrop = preprocess_input(cropimg)
  geteye=leyeCascade.detectMultiScale(cropimg,scaleFactor = 1.1, minNeighbors = 4)
  gozler=[]
  for (x,y,w,h) in geteye:
      gozler.append(cropimg[y:y+h, x:x+w])
  try:
    imageEye = Image.fromarray(gozler[0].astype(np.uint8))
  except:
    imageEye=cropimg
    print("File with no eye region: "+ filename)

  imageEye = np.expand_dims(imageEye, axis=0)
  imageEye = preprocess_input(imageEye)
  predictions=new_model.predict(imageEye)
  return reversed_dictionary.get(np.argmax(predictions))






In [ ]:
def get_prediction_yawn(filename,model):
  img = image.load_img(filename,color_mode='rgb', target_size=(256, 256))
  x = image.img_to_array(img)
  # Adding the fouth dimension, for number of images
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  prediction_yawn = model.predict(x)
  return reversed_dictionary.get(np.argmax(prediction_yawn))


In [ ]:
fileList=["/content/gdrive/MyDrive/drowsy_detection/test_images/frame2260.jpg","/content/gdrive/MyDrive/drowsy_detection/test_images/frame150.jpg","/content/gdrive/MyDrive/drowsy_detection/test_images/Data_frames_video/Data_frames/drowsy_data/frame400.jpg"]

In [ ]:
dir_list = os.listdir('/content/gdrive/MyDrive/drowsy_detection/test_images/Data_frames_video/Data_frames/drowsy_data')
fileFolder = '/content/gdrive/MyDrive/drowsy_detection/test_images/Data_frames_video/Data_frames/drowsy_data/'

In [ ]:
len(dir_list)

265

In [ ]:
import time
start = time.time()

predEyeClosedCount = 0
predYawn = 0
i=0
for filename in dir_list:
  print(filename,i)
  predictionEyeRegion = get_prediction_eye(fileFolder+filename,new_model)
  if predictionEyeRegion == 'Closed':
    predEyeClosedCount+=1

  predictionYawn = get_prediction_yawn(fileFolder+filename,new_model)
  if predictionYawn == 'yawn':
      predYawn+=1

  if predictionEyeRegion == predictionYawn:
    predEyeClosedCount-=1
  i+=1
  #print(predictionEyeRegion,predictionYawn)

print("--------------------------------------------")
print("Number of closed eye time: "+str(predEyeClosedCount))
print("Number of yawns: "+ str(predYawn))
print("-------------------------------------------")
print(f'Time: {time.time() - start}')


frame400.jpg 0
frame190.jpg 1
frame0.jpg 2
frame60.jpg 3
frame90.jpg 4
frame40.jpg 5
frame80.jpg 6
frame20.jpg 7
frame50.jpg 8
frame100.jpg 9
frame30.jpg 10
frame70.jpg 11
frame10.jpg 12
frame150.jpg 13
frame290.jpg 14
frame270.jpg 15
frame220.jpg 16
frame300.jpg 17
frame140.jpg 18
frame280.jpg 19
frame250.jpg 20
frame380.jpg 21
frame310.jpg 22
frame410.jpg 23
frame240.jpg 24
frame260.jpg 25
frame320.jpg 26
frame170.jpg 27
frame130.jpg 28
frame200.jpg 29
frame230.jpg 30
frame350.jpg 31
frame180.jpg 32
frame160.jpg 33
frame370.jpg 34
frame110.jpg 35
frame330.jpg 36
frame340.jpg 37
frame390.jpg 38
frame120.jpg 39
frame210.jpg 40
frame360.jpg 41
frame600.jpg 42
frame430.jpg 43
frame720.jpg 44
frame460.jpg 45
frame490.jpg 46
frame630.jpg 47
frame510.jpg 48
frame550.jpg 49
frame500.jpg 50
frame580.jpg 51
frame590.jpg 52
frame450.jpg 53
frame480.jpg 54
frame690.jpg 55
frame670.jpg 56
frame640.jpg 57
frame610.jpg 58
frame650.jpg 59
frame420.jpg 60
frame660.jpg 61
frame560.jpg 62
frame440.jpg 

In [ ]:
predEyeClosedCount

205

In [ ]:
predYawn

35